# Deep Learning: Sequence Data - NLP
> AHSNCCU/NTNU CSIE 王修佑

在本教學中，我們暫不討論單獨的資料片段 (例如靜止影像)，而是關注在需要與一系列資料項目共同詮釋才能成立的資料。我們將使用文字句子來做為範例。語言是與生俱來的序列資料，以字母組成單詞，再以單詞組成句子。其他序列資料的範例包括股票價格和會隨時間變化的天氣資料。而由一系列靜止影像組成的影片，也是一種序列資料。面對這種個資料中的元素間有前後序列關係的資料，我們需要以不同的方式來處理。

## 目標

* 準備好要用在[遞歸神經網路](https://developers.google.com/machine-learning/glossary#recurrent-neural-network) (RNN) 中的序列資料
* 建立並訓練能進行文字預測的模型

## 標題生成器

我們都曾在應用程式中看過文字預測功能，例如在搜尋列、手機或文字編輯器中自動完成句子。許多優秀的文字預測模型都是在超大型資料集上進行訓練，而這需要大量的時間和/或處理能力來進行訓練。在本練習中，我們將透過實作的預測功能，展示一些簡單的概念關於語言處理、序列資料，以及經典架構，*遞歸神經網路*(*RNN*)。

## 讀取資料

我們的資料集由數個月的[紐約時報](https://www.nytimes.com/)報導標題所組成。我們首先會讀取所有報導中的標題。這些文章為 CSV 檔案，因此我們可以使用 *Pandas* 來讀取它們。

In [ ]:
import os 
import pandas as pd

nyt_dir = 'nyt_articles/'

all_headlines = []
for filename in os.listdir(nyt_dir):
    if 'Articles' in filename:
        # Read in all all the data from the CSV file
        headlines_df = pd.read_csv(nyt_dir + filename)
        # Add all of the headlines to our list
        all_headlines.extend(list(headlines_df.headline.values))
len(all_headlines)

現在讓我們看看前幾個標題：

In [ ]:
all_headlines[:20]

## 清理資料

自然語言處理 (NLP) 工作 (也就是以電腦處理語言) 其中一個關鍵部分就是先以電腦可理解的方式處理文字。我們會將資料集內出現的每個字各自轉換成一個數值。這是所謂*標記化*(TTokenization)過程的一部分。

在此之前，我們需要先確保資料品質完好。有些標題被列為「Unknown」。我們不想要這些項目出現在訓練集中，因此要先過濾掉它們：

In [ ]:
# Remove all headlines with the value of "Unknown"
all_headlines = [h for h in all_headlines if h != "Unknown"]
len(all_headlines)

再來看看下一個：

In [ ]:
all_headlines[:20]

我們也想移除標點符號，並將所有的字母設為小寫好讓模型更容易訓練。一個句子以「!」或「?」結尾，或單詞是否大寫 (如大寫「The」或小寫「the」) 對我們模型的用途來說並沒有太大的差別。減少特殊標記能讓模型更容易訓練。

在標記化前我們可以先自行過濾句子，但因為我們可以使用 Keras `Tokenizer`來處理，因此目前不需要這樣做。

## 標記化(Tokenization)

現在，我們的資料集由一組標題組成，每個標題則由一系列單詞組成。我們希望為模型提供某種能讓它理解這些單詞的表現方式。在標記化之後，我們會將一段文字分成較小的區塊 (標記 or Token)，在此範例中為單詞。每個獨一無二的單詞都會被賦予一個數值，好讓我們的模型可以理解資料。Keras 有一個可協助我們標記化資料的類別：

```python
tf.keras.preprocessing.text.Tokenizer(
    num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,
    split=' ', char_level=False, oov_token=None, document_count=0, **kwargs
)
```

請看看在 Keras 中的 [Tokenizer](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer) 類別，裡面的預設值已設為可用於處理我們的案例。`filters`字串刪除了標點符號，而 `lower` 旗標則將單詞設為小寫。

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Tokenize the words in our headlines
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_headlines)
total_words = len(tokenizer.word_index) + 1
print('Total words: ', total_words)

我們能快速流覽 `word_index`字典來瞭解 Tokenizer 是如何保存單詞的：

In [ ]:
# Print a subset of the word_index dictionary created by Tokenizer
subset_dict = {key: value for key, value in tokenizer.word_index.items() \
               if key in ['a','man','a','plan','a','canal','panama']}
print(subset_dict)

我們也能使用 `texts_to_sequences`方法來瞭解 Tokenizer 是如何保存單詞的：

In [ ]:
tokenizer.texts_to_sequences(['a','man','a','plan','a','canal','panama'])

## 建立序列

現在我們完成資料標記，將每個單詞各自轉換為代表數字，接下來我們要用標題建立標記的序列。我們將用這些序列來訓練深度學習模型。

例如，讓我們先來看看這個標題：「nvidia launches ray tracing gpus」。每個單詞都會替換為對應的數值，例如：nvidia - 5、launches - 22、ray - 94、tracing - 16、gpus - 102。完整序列為：[5、22、94、16、102]。不過，標題中較小型的序列也能用來訓練，例如「nvidia launches」。我們將用每個標題建立一組序列來匯入我們的資料集。接下來，讓我們使用 Tokenizer 來將標題轉換成一組序列吧。

In [ ]:
# Convert data to sequence of tokens 
input_sequences = []
for line in all_headlines:
    # Convert our headline into a sequence of tokens
    token_list = tokenizer.texts_to_sequences([line])[0]
    
    # Create a series of sequences for each headline
    for i in range(1, len(token_list)):
        partial_sequence = token_list[:i+1]
        input_sequences.append(partial_sequence)

print(tokenizer.sequences_to_texts(input_sequences[:5]))
input_sequences[:5]

## 填充序列

現在我們序列的長度各不相同。要讓模型能夠訓練資料，我們需要將所有序列的長度保持相同。為了達到這個目的，我們要為序列加上填充。我們可以使用 Keras 內建的 `pad_sequences`[方法](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences)。

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Determine max sequence length
max_sequence_len = max([len(x) for x in input_sequences])

# Pad all sequences with zeros at the beginning to make them all max length
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
input_sequences[0]

## 建立預測指標與目標

我們也需將序列分為預測指標和目標。序列中的最後一個單詞會是我們的目標，而序列中的第一個單詞則為預測指標。舉例來說，完整的標題是：「nvidia releases ampere graphics cards」

<table>
<tr><td>預測標指標</td> <td>目標</td></tr>
<tr><td>nvidia                   </td> <td>  releases </td></tr>
<tr><td>nvidia releases               </td> <td>  ampere </td></tr>
<tr><td>nvidia releases ampere      </td> <td>  graphics</td></tr>
<tr><td>nvidia releases ampere graphics </td> <td>  cards</td></tr>
</table>

In [ ]:
# Predictors are every word except the last
predictors = input_sequences[:,:-1]
# Labels are the last word
labels = input_sequences[:,-1]
labels[:5]

與先前的章節相同，這些目標是可以分類的。我們從可能的總詞彙中預測出一個單詞。我們會讓網路預測所有類別字彙中的單一類別字彙，而不是預測純量數值。

In [ ]:
from tensorflow.keras import utils

labels = utils.to_categorical(labels, num_classes=total_words)

## 建立模型

在我們的模型中，我們將使用幾個新的分層來處理我們的連續資料。

### [嵌入層](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding)

我們的第一層是嵌入層：

```Python
model.add(Embedding(input_dimension, output_dimension, input_length=input_len))
```

該層將讀取經標記後的序列，並在訓練資料集中學習所有單詞的嵌入。從數學上來說，嵌入層的運作方式與神經網路中的神經元相同，但從概念上來說，其目標是減少部分或所有功能的維度數量。在這種情況下，它會將每個單詞轉換為向量，而該向量中的資訊將包含每個單詞之間的關係。

你可以在[這裡](https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/)瞭解更多關於嵌入層的資訊。

<img src="./images/embedding.png" style="width: 300px;">

### [長短期記憶層](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM)

下一個且非常重要的分層是長短期記憶層 (LSTM)。長短期記憶是一種遞歸神經網路 (RNN)。與我們之前看到過的傳統向前傳遞神經網路 (feed-forward NN) 不同，遞歸網路內部具有迴圈，得以反覆使用資訊。以下是遞歸網路表現方式的範例：

<img src="./images/rnn_rolled.png" style="width: 150px;">

新資訊 (x) 會傳到網路上，產生出預測值 (h)。此外，該分層中的資訊也會被儲存下來，做為下個預測的輸入。這看起來有點複雜，但讓我們來仔細看看：

<img src="./images/rnn_unrolled.png" style="width: 600px;">

我們可以看到，當將新的資料 (x) 送入網路時，此網路不但會產出預測 (h)，也會將部分資訊傳遞到下一層。下一層會獲得另一段資料，但同時也能從上一層學習。

傳統遞歸神經網路的問題是較新的資料對於最後預測的影響力，會較先前資料更大。長短期記憶 (LSTM) 則是一種特殊種類的遞歸層，能夠學習和保留較長期的資訊。若要深入瞭解遞歸神經網路和長短期記憶，建議閱讀[這篇文章](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)。

好，我們開始建立模型吧：

In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential

# Input is max sequence length - 1, as we've removed the last word for the label
input_len = max_sequence_len - 1 

model = Sequential()

# Add input embedding layer
model.add(Embedding(total_words, 10, input_length=input_len))

# Add LSTM layer with 100 units
model.add(LSTM(100))
model.add(Dropout(0.1))

# Add output layer
model.add(Dense(total_words, activation='softmax'))

In [ ]:
model.summary()

## 編寫模型

就像先前一樣，由於我們是分類性地從總辭彙中預測單詞，我們將使用分類式交叉熵演算法來編寫模型。在此情況下，由於文字預測與影像分類不同，不是根據準確與否來衡量，因此我們將不使用精準度作為指標。

我們也會選擇適合長短期記憶工作的特定最佳化工具，也就是 *Adam* 最佳化工具。關於最佳化工具的細節解說有點超出了本課程的範圍，但總之最重要的是瞭解在面對不同的深度學習工作時，能夠使用不同的最佳化工具。你可以在[這裡](https://medium.com/datadriveninvestor/overview-of-different-optimizers-for-neural-networks-e0ed119440c3)閱讀更多相關資訊，包括 Adam 最佳化工具。

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

## 訓練模型

與前面的各章節類似，我們將以相同的方式擬合模型。這次我們將訓練 30 個 Epoch，這將需要幾分鐘的時間。你會發現在這個例子下，我們沒有訓練或驗證準確度分數。這反映出文字預測所要解決的問題類型是不同的。

In [ ]:
model.fit(predictors, labels, epochs=30, verbose=1)

## 結果討論

可以看出隨著訓練過程的進行，損失也在減少。我們可以進一步訓練模型來減少損失，但這需要花一些時間，而現在的目標並不是建立一個完美的文字預測器。接下來，我們試著使用模型來進行預測吧。

## 進行預測

為了進行預測，我們需要從種子文字開始，並使用與準備資料集相同的方式來處理。這表示我們需要進行標記化與填充。完成後，我們便能將種子文字傳送到模型中進行預測。我們將建立一個稱為 `predict_next_token`的函數來預測：

In [ ]:
def predict_next_token(seed_text):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    prediction = model.predict_classes(token_list, verbose=0)
    return prediction

In [ ]:
prediction = predict_next_token("today in new york")
prediction

接著使用 Tokenizer 來解碼預測出的單詞：

In [ ]:
tokenizer.sequences_to_texts([prediction])

## 產生新標題

現在能夠預測新的單詞了，接下來讓我們建立一個函數來預測由超過一個單詞所組成的標題。下面的函數會建立一個任意長度的新標題。

In [ ]:
def generate_headline(seed_text, next_words=1):
    for _ in range(next_words):
        # Predict next token
        prediction = predict_next_token(seed_text)
        # Convert token to word
        next_word = tokenizer.sequences_to_texts([prediction])[0]
        # Add next word to the headline. This headline will be used in the next pass of the loop.
        seed_text += " " + next_word
    # Return headline as title-case
    return seed_text.title()

用幾個標題試試看吧！

In [ ]:
seed_texts = [
    'washington dc is',
    'today in new york',
    'the school district has',
    'crime has become']
for seed in seed_texts:
    print(generate_headline(seed, next_words=5))

在經過 30 Epoch 的訓練後，預測結果可能會看起來有些靠不住。我們能注意到大多數的標題在文法上基本正確，但上下文的邏輯卻不一定通順。但這可能在執行更多 Epoch 後會稍有改善。你可以透過再次執行訓練 `fit`儲存格 (反覆執行！) 來改善，每次訓練 30 Epoch。應該就能看到損失率逐漸降低。接著再進行一次測試。結果可能會有不小的變化！

而其他的改善方式則是嘗試以 Word2Vec 或 GloVe 使用預先訓練嵌入，而不是在訓練時透過 Keras 嵌入層進行學習。你可以在[這裡](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html)找到相關方法。

儘管如此，現今自然語言處理的發展，已超越了簡單的長短期記憶模型，發展成以轉譯器為基礎的預先訓練模型，而這些模型能夠從如維基百科等的大量文字資料中學習語言脈絡。這些預先訓練的模型則用作遷移學習的起始點來解決自然語言處理工作，如我們剛才嘗試過的文字自動完成功能。你可以藉由以 [GPT-2 模型](https://openai.com/blog/better-language-models/)為基礎所發展出來的[世上最先進的文字預測器](https://transformer.huggingface.co/doc/gpt2-large)來嘗試。

若要深入瞭解以轉譯器為基礎的模型，請參閱這則有關雙向編碼器表示法 (BERT)的[貼文](https://ai.googleblog.com/2018/11/open-sourcing-bert-state-of-art-pre.html)，並在此 DLI 課程中的「下一步」頁面中尋找更多額外的課程資訊。

## 摘要

做得好！你已成功訓練模型以預測標題中的單詞，並使用該模型建立各種長度的標題。歡迎你隨時試用並產出更多標題。

### 清除記憶體
在繼續之前，請執行下列儲存格以清除 GPU 記憶體。

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## 下一步

恭喜，你已完成了本課程中的所有學習目標！

作為最後的練習並取得課程認證，請成功完成在[評估](07_assessment.ipynb)中的端對端影像分類問題。